This notebook introduces the h2ohyperopt module for Python. Here we begin with using single model optimizers and introduce the functions available to a user.

In [2]:
#Imports
import h2o
import h2ohyperopt

In [3]:
#Initializing h2o
h2o.init()

H2O cluster uptime:,49 seconds 623 milliseconds
H2O cluster version:,3.8.3.3
H2O cluster name:,H2O_started_from_python_abhishek_bdx287
H2O cluster total nodes:,1
H2O cluster total free memory:,1.76 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


### Data Processing
The test dataset used for demonstrating the capabilities of H2OHyperopt is the titanic dataset. The function ```data()``` is used to preprocess the dataset.

In [4]:
def data():
    """
    Function to process the example titanic dataset.
    Train-Valid-Test split is 60%, 20% and 20% respectively.
    Output
    ---------------------
    trainFr: Training H2OFrame.
    testFr: Test H2OFrame.
    validFr: Validation H2OFrame.
    predictors: List of predictor columns for the Training frame.
    response: String defining the response column for Training frame.
    """
    titanic_df = h2o.import_file(path="https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")

    # Basic preprocessing
    # columns_to_be_used - List of columns which are used in the training/test
    # data
    # columns_to_factorize - List of columns with categorical variables
    columns_to_be_used = ['pclass', 'age', 'sex', 'sibsp', 'parch', 'ticket',
                          'embarked', 'fare', 'survived']
    columns_to_factorize = ['pclass', 'sex', 'sibsp', 'embarked', 'survived']
    # Factorizing the columns in the columns_to_factorize list
    for col in columns_to_factorize:
        titanic_df[col] = titanic_df[col].asfactor()
    # Selecting only the columns we need
    titanic_frame = titanic_df[columns_to_be_used]
    trainFr, testFr, validFr = titanic_frame.split_frame([0.6, 0.2],
                                                         seed=1234)
    predictors = trainFr.names[:]
    # Removing the response column from the list of predictors
    predictors.remove('survived')
    response = 'survived'
    return trainFr, testFr, validFr, predictors, response

In [5]:
trainFr, testFr, validFr, predictors, response = data()


Parse Progress: [##################################################] 100%


### Single Model Type Based Optimization

Let us demonstrate the GBMOptimizer and extend the example to DLEOptimizer. Since the problem is a binary classification problem, we specify the metric to AUC.

In [6]:
model_gbm = h2ohyperopt.GBMOptimizer(metric='auc')

Now we need to fix the search space for the models. H2ohyperopt has built in default search spaces.

In [7]:
#To find the searchable parameters
model_gbm.list_all_parameters()

['learn_rate',
 'fold_column',
 'response_column',
 'stopping_metric',
 'score_tree_interval',
 'sample_rate_per_class',
 'seed',
 'weights_column',
 'nfolds',
 'max_after_balance_size',
 'offset_column',
 'checkpoint',
 'stopping_tolerance',
 'training_frame',
 'col_sample_rate_per_tree',
 'max_runtime_secs',
 'max_abs_leafnode_pred',
 'quantile_alpha',
 'balance_classes',
 'learn_rate_annealing',
 'sample_rate',
 'validation_frame',
 'max_depth',
 'model_id',
 'build_tree_one_node',
 'ntrees',
 'min_split_improvement',
 'ignored_columns',
 'tweedie_power',
 'min_rows',
 'score_each_iteration',
 'max_confusion_matrix_size',
 'col_sample_rate_change_per_level',
 'nbins',
 'histogram_type',
 'col_sample_rate',
 'fold_assignment',
 'keep_cross_validation_predictions',
 'stopping_rounds',
 'nbins_top_level',
 'max_hit_ratio_k',
 'nbins_cats',
 'r2_stopping',
 'distribution',
 'class_sampling_factors',
 'ignore_const_cols',
 'keep_cross_validation_fold_assignment']

#### Distributions available
For the search space, we need to pick out distributions which are sampled in order for the TPE's generate the next point to be evaluated. The information about these functions can be extracted using the function ```show_distributions_info()```.

In [8]:
h2ohyperopt.show_distributions_info()

List of Distributions available
Distribution name : uniform
Docstring
------------------------------------

    Function to create hyperopt uniform random variable
    Input
    ------------------
    name - Variable name
    (lower, upper) - Tuple with lower and upper bound
    
Distribution name : randint
Docstring
------------------------------------

    Function to create hyperopt randint variable
    Input
    ------------------
    name - Variable name
    (lower, upper) - Tuple with lower and upper bound
    
Distribution name : choice
Docstring
------------------------------------

    Function to create hyperopt choice variable
    Input
    ------------------
    name - Variable name
    choice_list - List with possible choices
    
Distribution name : loguniform
Docstring
------------------------------------

    Function to create hyperopt loguniform variable
    Input
    ------------------
    name - Variable name
    (lower, upper) - Tuple of bounds.

    Distribution l

#### Parameter specification format

The parameters are to be provided in the format (distributions,(parameters)). The distribution name is to be a string. This is parsed internally to create hyperopt sampling objects. In order to use the default optimization space use the argument ```'Default'```.

Each of the Model optimizers has a default search space. To access the default search space, the attribute ```def_params``` is provided. The Optimizers store these default parameters as a dictionary.

In [9]:
model_gbm.def_params

{'col_sample_rate': ('uniform', (0.5, 0.8)),
 'learn_rate': ('uniform', (0.01, 0.2)),
 'max_depth': ('randint', (2, 20)),
 'metric': 'auc',
 'nfolds': 5,
 'ntrees': ('choice', [100, 200]),
 'sample_rate': ('uniform', (0.8, 1.0)),
 'stopping_rounds': 10}

To select the optimization parameters, the function ```select_optimization_parameters``` must be called with the search space. An example is demonstrated below.

In [10]:
# To use the default search space
# model_gbm.select_optimization_parameters("Default")
# To use a combination of Default parameters and the customized parameters.
model_gbm.select_optimization_parameters({'col_sample_rate': 'Default',
                                          'ntrees': 200,
                                          'learn_rate': ('uniform',(0.05, 0.2)),
                                          'nfolds': 7})

If you want to add more parameters, the function ```add_optimization_parameters``` is provided. The input format is still dictionary based. The function ```rem_optimization_parameters``` can be used to remove parameters after initialization. Multiple search variables can be removed by inputting a list of unwanted variables.

In [11]:
model_gbm.add_optimization_parameters({'sample_rate':'Default'})
model_gbm.rem_optimization_parameters('sample_rate')

Starting the evaluation of models. Let the maximum number of models to be evaluated be 100. The validation frame is necessary for the model to score the model and optimize the hyperparameters effectively. The function ```start_optimization``` is starts the evaluation of models.

In [12]:
model_gbm.start_optimization(num_evals=10, trainingFr=trainFr,
                         validationFr=validFr, response=response,
                         predictors=predictors)


gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%


To get the best model from the optimization process, the attribute ```best_model``` can be accessed.

In [13]:
model_gbm.best_model 

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1470427337422_19338

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,200.0,200.0,51352.0,5.0,5.0,5.0,7.0,26.0,15.475




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0216659767126
R^2: 0.908574894874
LogLoss: 0.0963158961938
Mean Per-Class Error: 0.0156125371933
AUC: 0.998352742078
Gini: 0.996705484155

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.484742133118: 


,0,1,Error,Rate
0,472.0,7.0,0.0146,(7.0/479.0)
1,5.0,296.0,0.0166,(5.0/301.0)
Total,477.0,303.0,0.0154,(12.0/780.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid
accuracy,0.8192521,0.0224962,0.7837838,0.8474576,0.8782608,0.8220339,0.8181818,0.8,0.7850468
auc,0.8662204,0.0160003,0.8668047,0.8511628,0.9057404,0.8832572,0.8665217,0.8624443,0.8276119
err,0.1807479,0.0224962,0.2162162,0.1525424,0.1217391,0.1779661,0.1818182,0.2,0.2149533
err_count,20.0,2.2990682,24.0,18.0,14.0,21.0,22.0,18.0,23.0
f0point5,0.7606705,0.0347349,0.6954887,0.8040201,0.8505155,0.7471265,0.7563025,0.7604563,0.7107843
f1,0.7781146,0.0243323,0.7551020,0.7804878,0.825,0.7878788,0.7659575,0.8163266,0.7160494
f2,0.7995433,0.0346950,0.8258929,0.7582939,0.8009709,0.8333333,0.7758621,0.8810573,0.7213930
lift_top_group,2.4011884,0.3398914,2.642857,2.7441862,2.7380953,2.6222222,2.6304348,2.0930233,1.3375
logloss,0.5323395,0.0785848,0.4968326,0.4750118,0.3985118,0.4623221,0.534689,0.5906845,0.7683242
max_per_class_error,0.2517831,0.0272322,0.2753623,0.2558140,0.2142857,0.2054795,0.2173913,0.3191489,0.275



Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error
,2016-08-05 13:04:36,4.915 sec,0.0,0.2369806,0.6668775,0.5,1.0,0.6141026
,2016-08-05 13:04:36,4.920 sec,1.0,0.2037706,0.5975941,0.8895713,2.5913621,0.1576923
,2016-08-05 13:04:36,4.924 sec,2.0,0.1797976,0.5472035,0.8972527,2.5913621,0.1564103
,2016-08-05 13:04:36,4.930 sec,3.0,0.1623485,0.5092148,0.9070253,2.5913621,0.1717949
,2016-08-05 13:04:36,4.936 sec,4.0,0.1515715,0.4844515,0.9073652,2.5913621,0.15
---,---,---,---,---,---,---,---,---
,2016-08-05 13:04:40,8.727 sec,142.0,0.0318240,0.1313323,0.9961402,2.5913621,0.0282051
,2016-08-05 13:04:40,8.774 sec,143.0,0.0317430,0.1310953,0.9961541,2.5913621,0.0282051
,2016-08-05 13:04:40,8.821 sec,144.0,0.0314308,0.1299551,0.9961818,2.5913621,0.0282051
,2016-08-05 13:04:40,8.867 sec,145.0,0.0313660,0.1297720,0.9962026,2.5913621,0.0282051



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
sex,197.1661072,1.0,0.3615609
pclass,80.5559006,0.4085687,0.1477225
age,80.4256668,0.4079082,0.1474836
fare,78.9291687,0.4003181,0.1447394
ticket,59.1445198,0.2999731,0.1084585
sibsp,25.3239841,0.1284398,0.0464388
embarked,14.3758030,0.0729121,0.0263622
parch,9.3981142,0.0476660,0.0172342


To get the parameters in order to reproduce the results, the function ```best_model_parameters``` is provided.

In [14]:
# To get the best model parameters
model_gbm.best_model_parameters()

{'col_sample_rate': 0.784432749752081,
 'learn_rate': 0.14752304082076237,
 'metric': 'auc',
 'nfolds': 7,
 'ntrees': 200}

To get the training and validation scores, use the function ```best_model_scores```

In [15]:
model_gbm.best_model_scores()

The training loss metric(auc) is : 0.998352742078
The validation loss metric(auc) is : 0.853508656006


To get the test score or score on any dataset, use the function ```best_model_test_scores()```

In [16]:
print "Test Dataset AUC : ", model_gbm.best_model_test_scores(testFr)

Test Dataset AUC :  0.796731473655


#### Ensembling multiple models

Ensembling can help improve performance. As a basic ensembler, we provide an option to ensemble the top n models in order to build a better model. The function ```best_model_ensemble``` creates and trains the ensemble based off the training data.

In [17]:
# To ensemble top 5 GBM models
model_gbm.best_model_ensemble(num_models=5)


gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%
Model Trained


To score any dataset with the ensemble, the function ```score_ensemble``` can be used.

In [18]:
#Test score of the ensembler model
model_gbm.score_ensemble(testFr)


gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%


0.7695407156945618

To access the ensembled model, we can access the attribute ```ensemble_model```

In [19]:
model_gbm.ensemble_model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1470427337422_23836

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,200.0,200.0,19665.0,0.0,4.0,1.32,1.0,5.0,2.63




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0127584283229
R^2: 0.946162563261
LogLoss: 0.0635964594194
Mean Per-Class Error: 0.0151859840892
AUC: 0.989845955375
Gini: 0.97969191075

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.437854535296: 


,0,1,Error,Rate
0,474.0,5.0,0.0104,(5.0/479.0)
1,6.0,295.0,0.0199,(6.0/301.0)
Total,480.0,300.0,0.0141,(11.0/780.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9859061,0.0085661,1.0,0.9936306,0.9738562,0.9691358,0.9929078
auc,0.9851784,0.0095407,1.0,0.9912801,0.9622474,0.9782609,0.9941038
err,0.0140939,0.0085661,0.0,0.0063694,0.0261438,0.0308642,0.0070922
err_count,2.2,1.371131,0.0,1.0,4.0,5.0,1.0
f0point5,0.9847368,0.0060570,1.0,0.9854015,0.9768212,0.9763314,0.9851301
f1,0.9824393,0.0101813,1.0,0.9908257,0.9672131,0.9635037,0.9906542
f2,0.9802703,0.0150436,1.0,0.9963099,0.9577922,0.9510086,0.9962406
lift_top_group,2.5891278,0.1470896,2.6935484,2.9074075,2.4211807,2.3142858,2.6092162
logloss,0.0863423,0.0407910,0.0186936,0.0487273,0.1591941,0.1516153,0.0534814
max_per_class_error,0.0253205,0.0161999,0.0,0.0097087,0.0483871,0.0571429,0.0113636



Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error
,2016-08-05 13:04:57,4.359 sec,0.0,0.2369806,0.6668775,0.5,1.0,0.6141026
,2016-08-05 13:04:57,4.363 sec,1.0,0.1944670,0.5777871,0.9879213,2.5913621,0.0153846
,2016-08-05 13:04:57,4.366 sec,2.0,0.1602600,0.5059243,0.9899014,2.5913621,0.0141026
,2016-08-05 13:04:57,4.370 sec,3.0,0.1327395,0.4468068,0.9899014,2.5913621,0.0141026
,2016-08-05 13:04:57,4.375 sec,4.0,0.1105539,0.3973801,0.9899014,2.5913621,0.0141026
---,---,---,---,---,---,---,---,---
,2016-08-05 13:05:01,7.898 sec,196.0,0.0127584,0.0635965,0.9898460,2.5913621,0.0141026
,2016-08-05 13:05:01,7.927 sec,197.0,0.0127584,0.0635965,0.9898460,2.5913621,0.0141026
,2016-08-05 13:05:01,7.955 sec,198.0,0.0127584,0.0635965,0.9898460,2.5913621,0.0141026
,2016-08-05 13:05:01,7.984 sec,199.0,0.0127584,0.0635965,0.9898460,2.5913621,0.0141026



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
predict3,534.0779419,1.0,0.5710807
predict,397.1284485,0.7435777,0.4246428
predict1,2.8051803,0.0052524,0.0029995
predict0,0.6348431,0.0011887,0.0006788
predict2,0.5593830,0.0010474,0.0005981
